In [16]:

import numpy as np
import pandas as pd 
from sklearn.preprocessing import minmax_scale
from sklearn.multioutput import RegressorChain
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import make_scorer






In [4]:
def mean_euclidean_error(y_true, y_pred):
    """
    mean euclidean error MEE
    """

    error = 0
    if len(y_true) == len(y_pred):
        for i in range(len(y_true)):
            error = error + np.sqrt(np.square(y_pred[i][0]
                                    - y_true[i][0])
                                    + np.square(y_pred[i][1]
                                    - y_true[i][1]))
        return error / len(y_true)

In [5]:

train_df = pd.read_csv('../data/ML-CUP20-TR .csv')
test_df = pd.read_csv('../data/ML-CUP20-TS.csv')

# Spilitting the targets and input features



In [7]:
train_x = train_df.iloc[:, :-2].to_numpy()
train_y = train_df.iloc[:, [10, 11]].to_numpy()

In [8]:
"""Splitting the dataset into the Training set and Test set"""

(x_dev, x_test, y_dev, y_test) = train_test_split(train_x, train_y,
        test_size=0.25, random_state=30)
(x_train, x_val, y_train, y_val) = train_test_split(x_dev, y_dev,
        test_size=0.25, random_state=30)

In [9]:
""" Fitting KNN to the Training set with the best parameters obtained form gridsearch"""

knn_model = Pipeline(steps=[('scale', StandardScaler(with_mean=True,
                     with_std=True)), ('estimator',
                     RegressorChain(KNeighborsRegressor(
    algorithm='auto',
    leaf_size=1,
    metric='minkowski',
    metric_params=None,
    n_jobs=None,
    n_neighbors=9,
    p=2,
    weights='distance',
    ), order=[0, 1]))]).fit(x_train, y_train)

In [57]:
n_neighbors =  list(range(1,20))
leaf_size   = list(range(1,15))


In [30]:
n_neighbors

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49]

In [85]:
knn_model = Pipeline(steps=[('scale', StandardScaler(with_mean=True,
                     with_std=True)), ('estimator',
                     RegressorChain(KNeighborsRegressor(n_neighbors=4),
                     order=[0, 1]))])

In [59]:
knn_params = {
    'estimator__base_estimator__n_neighbors': n_neighbors,
    'estimator__base_estimator__leaf_size': leaf_size,
    'estimator__base_estimator__weights': ['uniform', 'distance'],
    'estimator__base_estimator__algorithm': ['auto', 'ball_tree',
            'kd_tree', 'brute'],
    'estimator__base_estimator__p':[1,2]
    }

In [35]:
mee = make_scorer(mean_euclidean_error, greater_is_better=False)

In [61]:
""" Gridsearch and Learning curve"""

for i in range(1):
    Grid = GridSearchCV(
        estimator=knn_model,
        param_grid=knn_params,
        scoring = mee,
        n_jobs=-1,
        cv=3,
        verbose=1,
        ).fit(x_train, y_train)
    
    
    print("Mean\tSTD\tParams")
    means = Grid.cv_results_['mean_test_score']
    stds = Grid.cv_results_['std_test_score']
    params = Grid.cv_results_['params']
    
    for mean, stdev, param in  (zip(means, stds, params)):
        print("%f (%f) with: %r" % (mean, stdev, param))
    
    y_pred = Grid.predict(x_val)
    y_pred_train = Grid.predict(x_train)
    y_tpred = Grid.predict(x_test)
    (train_sizes, train_scores, test_scores) = \
        learning_curve(Grid.best_estimator_, x_train, y_train, cv=3,
                       n_jobs=-1)
    df = pd.DataFrame(list(zip(means, stds, params)),  columns =['means', 'stds', 'params']).sort_values(by=['means'])
    df
    print(f"Best parameters:{Grid.best_params_}")
    print(f"Best training accuracy:{Grid.best_score_}")
    print(f"VR dataset accuracy score for best params:{r2_score(y_val, y_pred)}" )
    print(f"Mean_euclidean_error for VR dataset :{mean_euclidean_error(y_val, y_pred)}" )
    print(f"Mean_euclidean_error for TR dataset :{mean_euclidean_error(y_train, y_pred_train)}" )
    print(f"Mean_euclidean_error for TS dataset :{mean_euclidean_error(y_test, y_tpred)}" )
    print(f"Best model :{Grid.best_estimator_}")
    print(f"Total time :{end - start}")
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    (fig, ax) = plt.subplots(1, figsize=(10, 10))
    ax.grid(linestyle='-', linewidth='0.5', color='black')
    ax.set_axisbelow(True)
    plt.plot(train_sizes, train_scores_mean, '--', color='r',
             label='Training score')
    plt.plot(train_sizes, test_scores_mean, 'o-', color='g',
             label='Cross-validation score')
    plt.title(f"Learning curve CSVR)")
    plt.xlabel('Training examples')
    plt.ylabel('Score')
    plt.legend(loc='Best')
    plt.tight_layout()
#     plt.savefig('CSVR')
    plt.show()


Fitting 3 folds for each of 4256 candidates, totalling 12768 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 560 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 4592 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done 8192 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-1)]: Done 12592 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done 12768 out of 12768 | elapsed:   46.3s finished


Mean	STD	Params
-4.125511 (0.141578) with: {'estimator__base_estimator__algorithm': 'auto', 'estimator__base_estimator__leaf_size': 1, 'estimator__base_estimator__n_neighbors': 1, 'estimator__base_estimator__p': 1, 'estimator__base_estimator__weights': 'uniform'}
-4.125511 (0.141578) with: {'estimator__base_estimator__algorithm': 'auto', 'estimator__base_estimator__leaf_size': 1, 'estimator__base_estimator__n_neighbors': 1, 'estimator__base_estimator__p': 1, 'estimator__base_estimator__weights': 'distance'}
-3.906588 (0.102518) with: {'estimator__base_estimator__algorithm': 'auto', 'estimator__base_estimator__leaf_size': 1, 'estimator__base_estimator__n_neighbors': 1, 'estimator__base_estimator__p': 2, 'estimator__base_estimator__weights': 'uniform'}
-3.906588 (0.102518) with: {'estimator__base_estimator__algorithm': 'auto', 'estimator__base_estimator__leaf_size': 1, 'estimator__base_estimator__n_neighbors': 1, 'estimator__base_estimator__p': 2, 'estimator__base_estimator__weights': 'd

NameError: name 'end' is not defined

In [44]:
fit(x_train, y_train, validation_data=(x_test, y_test))

ValueError: Pipeline.fit does not accept the validation_data parameter. You can pass parameters to specific steps of your pipeline using the stepname__parameter format, e.g. `Pipeline.fit(X, y, logisticregression__sample_weight=sample_weight)`.

In [89]:
prova = [
 {'estimator__base_estimator__algorithm': 'auto', 'estimator__base_estimator__leaf_size': 1, 'estimator__base_estimator__n_neighbors': 12, 'estimator__base_estimator__p': 2, 'estimator__base_estimator__weights': 'distance'},
 {'estimator__base_estimator__algorithm': 'ball_tree', 'estimator__base_estimator__leaf_size': 10, 'estimator__base_estimator__n_neighbors': 15, 'estimator__base_estimator__p': 2, 'estimator__base_estimator__weights': 'distance'},
 {'estimator__base_estimator__algorithm': 'ball_tree', 'estimator__base_estimator__leaf_size': 10, 'estimator__base_estimator__n_neighbors': 12, 'estimator__base_estimator__p': 2, 'estimator__base_estimator__weights': 'distance'},
 {'estimator__base_estimator__algorithm': 'ball_tree', 'estimator__base_estimator__leaf_size': 10, 'estimator__base_estimator__n_neighbors': 11, 'estimator__base_estimator__p': 2, 'estimator__base_estimator__weights': 'distance'},
 {'estimator__base_estimator__algorithm': 'ball_tree', 'estimator__base_estimator__leaf_size': 9, 'estimator__base_estimator__n_neighbors': 14, 'estimator__base_estimator__p': 2, 'estimator__base_estimator__weights': 'uniform'}
]

In [68]:
df.iloc[-10:].means.unique()

array([-2.96412092])

In [90]:
mean_tr_loss = []
mean_vl_loss = [2.964121 , 2.996065, 2.964121,2.968047, 2.992259]
mean_ts_loss = []

In [91]:
for params in prova:
    knn_model = Pipeline(steps=[('scale', StandardScaler(with_mean=True,
                        with_std=True)), ('estimator',
                        RegressorChain(KNeighborsRegressor(algorithm=params['estimator__base_estimator__algorithm'], leaf_size=params['estimator__base_estimator__leaf_size'],
                                                       metric_params=None,n_jobs=None,
                                                       n_neighbors=params['estimator__base_estimator__n_neighbors'], weights=params['estimator__base_estimator__weights'],
                                                       p=params['estimator__base_estimator__p'],
                                                       )))]).fit(x_train, y_train)
    train_pred = knn_model.predict(x_train)
    mean_tr_loss.append(mean_euclidean_error(train_pred, y_train))
    internal_test_pred = knn_model.predict(x_test)
    mean_ts_loss.append(mean_euclidean_error(internal_test_pred, y_test))

In [ ]:
-2.96804673, -2.96804673, -2.96584517, -2.96412092,
       -2.9641209

In [92]:
print("mean_tr_loss\tmean_vl_loss\tmean_ts_loss")
for tr_loss, vl_loss, ts_loss in zip(mean_tr_loss, mean_vl_loss, mean_ts_loss):
    print("%f (%f) with: %f" % (tr_loss, vl_loss, ts_loss))

mean_tr_loss	mean_vl_loss	mean_ts_loss
0.000000 (2.964121) with: 2.673342
0.000000 (2.996065) with: 2.649795
0.000000 (2.964121) with: 2.673342
0.000000 (2.968047) with: 2.664215
2.711976 (2.992259) with: 2.663994


In [ ]:
{'estimator__base_estimator__algorithm': 'auto', 'estimator__base_estimator__leaf_size': 2, 'estimator__base_estimator__n_neighbors': 12, 'estimator__base_estimator__p': 2, 'estimator__base_estimator__weights': 'distance'}